<a href="https://colab.research.google.com/github/luciansantana/olympic-games/blob/master/Olimpiadas.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## O banco de dados utilizado neste notebook, se trata de informações dos atletas que disputaram os Jogos Olímpicos durante anos de 1986 a 2016.

# Importando bibliotecas

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns

import matplotlib as plt
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split 
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler
import statsmodels.api as sm
from sklearn import neighbors
from sklearn.metrics import mean_squared_error 
from math import sqrt

# Lendo e conhecendo o banco de dados

In [ ]:
athlets = pd.read_csv("https://raw.githubusercontent.com/luciansantana/olympic-games/master/athlete_events.csv?token=AO27OWE4WTSMJ7U6ALCDJXC63LZV4")
athlets.head()

Temos então uma tabela em que mostra total de medalhas conquistadas por atleta em cada nação.

In [ ]:
medalist = athlets.dropna(subset = ["Medal"])
medalist['Team'].value_counts()

In [ ]:
medalist['Team'].value_counts()[:20].plot(kind ='barh',figsize = (22,8))

Vale destacar a soberania da nação americana diantes os outros. Outro ressalva em função com a Federação da Rússia, por questões politicas e historicas, a nação disputou em momentos diferentes da história com dois nomes em vigencia: Russia(RUS) e Soviet Unian(URS).

Houve então, um agrupamento dos atletas que levam o nome de Soviet Union e Russia, para Russia Federation

In [ ]:
#athlets['Team'] = athlets['Team'].replace(to_replace = ['Soviet Union','Russia'],value = 'Russia Federation')
#athlets['NOC'] = athlets['NOC'].replace(to_replace = ['URS'],value = 'RUS')


Com isso a União Federativa da Rússia chegando a 3668 atletas com medalhas conquistadas, 1969 a menos que o Estados Unidos e 1503 a mais que a Alemanha 

In [ ]:
medalist_ajus = athlets.dropna(subset=['Medal'])
medalist_ajus['NOC'].value_counts()[:10]

In [ ]:
athlets['City'].value_counts()[:20]

 A cidade em que mais foi palco de atrações olímpicas foi Londres (Inglaterra), e em seguida Athina (Grécia) e Sidney (Austrália)

In [ ]:
medalist['Name'].value_counts()[:10].plot(kind = 'barh',figsize = (22,8))

O atleta com mais medalhas na historia das olimpiadas, é Michael Fred Phelps, nadador que até então se encontra aposentado. Disputou as olimpiadas entre as edições de 2000 a 2016

In [ ]:
phelps = athlets[athlets['Name']=='Michael Fred Phelps, II']
phelps['Medal'].value_counts()

Michael Phelps disputou 30 medalhas, e conquistou 28 medalhas, o que mostra acaba mostrando a eficiencia do atleta americano. Das medalhas conquistadas, 23 são de ouro, o que representa o total de 82.14%.

# Análise histórica

Como os dados estão dispostos por atletas, foi preciso criar funções que conseguissem captar a quantidade de medalhas conquistadas pela nação, mas nestas funções foi constatado a quantidade de medalhas por eventos, pois utilizando os dados apenas por atleta ou por esporte, dois atletas diferentes poderiam ganhar a mesma medalha devido aos esportes coletivos, e assim inflando a quantidade de medalhas, com isso nações que são mais vencedoras nos esportes coletivos levariam vantagem no quesito medalhas. Com isso vamos trabalhar a partir de agora com os dados conquistados por eventos.

In [ ]:
def medalseason(noc):
  data0 = athlets[athlets['NOC']==noc]
  datax = athlets[athlets['Season']=='Summer']
  #datax = athlets
  data0 = data0[data0['Season']=='Summer']
  data0_med = data0.dropna(subset = ['Medal'])
  year = datax.Year.unique()
  year = sorted(year)
  data2 = []
  numero1 = []
  for i in year:
    data1 = data0_med[data0_med['Year']==i]
    datax = data0[data0['Year']==i]
    medalhas = data1.Medal.unique()
    numero = len(datax.Name.unique())
    numero1.append(numero)
    medal2 = []
    for j in medalhas:
      medal0 = data1[data1['Medal']==j]
      medal1 = medal0.Event.unique().shape[0]
      medal2.append(medal1)
    soma = sum(medal2)
    data2.append(soma)
  nomes = [noc]*len(year)
  obj = {'Year': year,'Medals' : data2,'NOC': [noc]*len(year),'Number' : numero1}
  data3 = pd.DataFrame(obj)

  return data3
def medalacum(noc):
  data0 = medalseason(noc)
  tamanho = range(1,data0.shape[0])
  #tamanho = range(1,len(data0))
  data1 = data0['Medals']
  data2 = data0['Number']
  for j in tamanho:
     #data0.iloc[j][1] = data0.iloc[j][1]+data0.iloc[j-1][1]
     data1[j]= data1[j]+data1[j-1]
     data2[j]= data2[j]+data2[j-1]
  data0['Medals']=data1
  data0['Number']=data2
  return data0

def geralacum():
  noc = athlets.NOC.unique()
  noc = sorted(noc)
  anos = athlets[athlets['Season']=='Summer']
  anos = anos.Year.unique()
  anos = sorted(anos)
  data1 = pd.DataFrame()
  for i in noc:
    data = medalacum(i)
    data = pd.DataFrame(data)
    data1 = pd.concat([data1,data],axis = 0)
  #data1.columns = noc
  #data1.index = anos
  return data1
def geralseason():
  noc = athlets.NOC.unique()
  noc = sorted(noc)
  anos = athlets[athlets['Season']=='Summer']
  anos = anos.Year.unique()
  anos = sorted(anos)
  data1 = pd.DataFrame()
  for i in noc:
    data = medalseason(i)
    data = pd.DataFrame(data)
    data1 = pd.concat([data1,data],axis = 0)
  #data1.columns = noc
  #data1.index = anos
  return data1

Um exemplo de como as funções retornam:

In [ ]:
tabelacum = geralacum()
tabelasea = geralseason()

Temos a lista então das nações que mais conquistaram medalhas.

In [ ]:
quadro_geral = tabelasea.groupby(['NOC'])['Medals'].apply(lambda x : x.astype('float').sum()).sort_values(ascending = False)[:10]
noc = sorted(quadro_geral.index)
plt.figure(figsize = (22,8))
for i in noc:

  teste0 = tabelacum[tabelacum['NOC']==i]
  teste1 = teste0.set_index(['Year'])
  teste2 = teste1[['Medals']]
  plt.plot( teste2,label = i)
  plt.ylabel('Medalhas')
  plt.xlabel('Anos')
  plt.legend()

In [ ]:
quadro_geral = tabelasea.groupby(['NOC'])['Medals'].apply(lambda x : x.astype('float').sum()).sort_values(ascending = False)[:10]
noc = sorted(quadro_geral.index)
plt.figure(figsize = (22,8))
for i in noc:
  teste0 = tabelasea[tabelasea['NOC']==i]
  teste1 = teste0.set_index(['Year'])
  teste2 = teste1[['Medals']]
  plt.scatter(x = teste0['Year'] ,y = teste2['Medals'], label = i,s = 200)
  plt.ylabel('Medalhas')
  plt.xlabel('Anos')
  plt.legend()

In [ ]:
barra = tabelasea[tabelasea['Number']!=0]
barra = barra[barra['NOC']!='EUN']
barra_1 = barra.groupby(['NOC'])['Number'].apply(lambda x : x.astype('float').mean()).sort_values(ascending = False)[:10]
barra_1 = pd.DataFrame(barra_1)
noc1 = barra_1.index
plt.figure(figsize = (22,8))
plt.bar(noc1,barra_1['Number'])
plt.ylabel('Média de partipantes')
plt.xlabel('Nações')


In [ ]:
data_prop = tabelacum
data_prop['prop'] = data_prop['Medals']/data_prop['Number']
x = data_prop[data_prop['Year']==2016]
x = x[x['Number']>1000]
x_1 = x.sort_values('prop',ascending=False)[:10]
noc = sorted(x_1['NOC'])
plt.figure(figsize=(22,8))
plt.bar(x_1['NOC'],x_1['prop'])

Utilizamos então as 10 nações +Brasil para fazer a analise e depois fazer uma estimação via regressão linear para cada uma das delas. Começamos então com um gráfico onde mostra a evolução ao longo das edições do número de medalhas acumuladas.

# BRASIL

## Regressão linear

In [ ]:
from sklearn.preprocessing import MinMaxScaler
economy =  pd.read_csv("https://raw.githubusercontent.com/luciansantana/olympic-games/master/economy.csv")
economy = economy[economy['Nacao']=='BRA']
bra_season = tabelasea[tabelasea['NOC']=='BRA']
bra_season = bra_season[bra_season['Year']>=1960]
bra_season['PIB.per.capita'] = economy['PIB.per.capita'].values

In [ ]:
trein = bra_season.drop(columns=['Year','NOC'])
X = trein.iloc[:,1:].values
Y = trein.iloc[:,:1].values

In [ ]:
modelo0 = LinearRegression()
modelo0.fit(X,Y)
mean_squared_error(Y,modelo0.predict(X))

In [ ]:
modelo0.predict(X)

In [ ]:
modelo1 = sm.OLS(Y,X)
modelo1 = modelo1.fit()
mean_squared_error(Y,modelo1.predict(X))